## Imports

In [1]:
import pandas as pd
import pyarrow.parquet as pq
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic")
from collections import Counter
import pprint
pp = pprint.PrettyPrinter(indent=4)
from functools import reduce

## Functions

In [95]:
def get_word_pct_dict(lst_word):
    word_pct_dict = {}
    for word in lst_word:
        word_pct_dict.update({word : len(df_air_pol[df_air_pol['contents_nouns_len>1'].apply(lambda x: word in x)])/len(df_air_pol)})
        sorted_word_pct_dict = sorted(word_pct_dict.items(), key=lambda x:x[1], reverse=True)
    return sorted_word_pct_dict

def flattened_list(lst):
    flattened_lst = [item for sublist in lst for item in sublist]
    return flattened_lst

## Read Files

In [3]:
df_qesg28 = pq.read_pandas('df_qesg28_final.pq').to_pandas()

In [4]:
df_qesg28.head(5)

,id,keyword,title,link,contents,y,contents_clean
0,5,삼성전자,"[OA칼럼] 삼성전자, D램 EUV 공정 적용 의미와 전망은",http://www.thelec.kr/news/articleView.html?idx...,파운드리 사업과 장비 사용 공유\n\n편집자 주 : 전자부품 전문미디어 디일렉은 미...,제외,파운드리 사업과 장비 사용 공유편집자 주 전자부품 전문미디어 디일렉은 미국 메모리 ...
1,13,삼성전자,"애플은 더 싸게 내놓고, 교체수요는 뚝… 위기의 삼성폰",https://news.chosun.com/site/data/html_dir/202...,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,제외,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹에 직격탄을 맞아 판...
2,18,삼성전자,5주년 맞은 삼성페이의 진화... 비대면 결제시장서 '기회',http://biz.newdaily.co.kr/site/data/html/2020/...,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,제외,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시s6 출시를 한달 여 앞둔 상황에...
3,19,삼성전자,"`경영권 불승계`…""삼성물산·전자, 기업가치 재평가 기대""",https://www.edaily.co.kr/news/read?newsId=0144...,[이데일리 김재은 기자] 미래에셋대우는 11일 삼성물산(028260) 삼성전자(00...,주주가치보호,미래에셋대우는 11일 삼성물산 삼성전자 등 삼성그룹주의 기업가치 재평가가 기대된다고...
4,30,삼성전자,이건희 회장 병상 6년… 이재용표 '뉴 삼성' 도약할까,https://www.seoul.co.kr/news/newsView.php?id=2...,李 회장 여전히 의식 없지만 자가 호흡\n\n▲ 이재용 삼성전자 부회장.\n\n연합...,소유구조,회장 여전히 의식 없지만 자가 호흡 이재용 삼성전자 부회장.이건희 삼성전자 회장이 ...


## 대기오염 뉴스만 추출

In [5]:
df_air_pol = df_qesg28[df_qesg28['y']=='대기오염'].reset_index(drop=True)

In [6]:
len(df_air_pol)

255

In [7]:
len

<function len(obj, /)>

## 명사들만 추출

In [8]:
df_air_pol['contents_nouns'] = df_air_pol['contents_clean'].apply(lambda x:mecab.nouns(x))

In [9]:
df_air_pol['nouns_counter'] = df_air_pol['contents_nouns'].apply(lambda x: Counter(x))

In [10]:
df_air_pol['nouns_no']= df_air_pol['contents_nouns'].apply(lambda x: len(x))

In [11]:
df_air_pol

,id,keyword,title,link,contents,y,contents_clean,contents_nouns,nouns_counter,nouns_no
0,11129,삼성전기,"내연기관 자동차 시대 끝?…“디젤차 사, 말아?”",http://www.enewstoday.co.kr/news/articleView.h...,"[이뉴스투데이 윤진웅 기자]\n\n#서울에 사는 직장인 박수남(가명, 41세) 씨는...",대기오염,상황이 이렇자 자동차 회사들 역시 국내와 수입을 막론하고 미래차 개발에 박차를 가하...,"[상황, 자동차, 회사, 국내, 수입, 미래, 차, 개발, 박차, 국내, 자동차, ...","{'상황': 1, '자동차': 5, '회사': 1, '국내': 2, '수입': 2,...",289
1,102294,GS건설,허태수 GS 회장의 '친환경 경영'…공허하게 들리는 이유는,http://isplus.joins.com/article/933/23810933.html,GS그룹의 새 수장인 허태수 회장은 그룹의 미래 설계를 그리며 '친환경 경영'을 강...,대기오염,gs그룹의 새 수장인 허태수 회장은 그룹의 미래 설계를 그리며 친환경 경영을 강조하...,"[그룹, 수장, 허태수, 회장, 그룹, 미래, 설계, 친환경, 경영, 강조, 포스트...","{'그룹': 2, '수장': 1, '허태수': 1, '회장': 6, '미래': 1,...",338
2,106248,한국금융지주,"한국전력공사, '인도네시아 화력발전소 건설 사업' 이사회서 의결 ...",http://www.econonews.co.kr/news/articleView.ht...,[이코노뉴스=조승환 기자] 한국전력공사가 '인도네시아 자바 9·10호기 화력발전소 ...,대기오염,한국전력공사가 인도네시아 자바 910호기 화력발전소 건설 사업 추진을 일단 멈추기로...,"[한국전력공사, 인도네시아, 자바, 호기, 화력, 발전소, 건설, 사업, 추진, 한...","{'한국전력공사': 2, '인도네시아': 2, '자바': 1, '호기': 1, '화...",57
3,150481,현대중공업지주,비산먼지 저감 살수차 운영,https://www.kbmaeil.com/news/articleView.html?...,"포스코와 현대제철(주), 동국제강(주), 에코시스템(주), (주)제철세라믹, (주)...",대기오염,포스코와 현대제철 동국제강 에코시스템 제철세라믹 세기 등 6개 회사들이 오는 31일...,"[포스코, 현대제철, 동국제강, 에코, 시스템, 제철, 세라믹, 세기, 등, 개, ...","{'포스코': 1, '현대제철': 1, '동국제강': 1, '에코': 1, '시스템...",36
4,155072,오뚜기,"‘대기오염 측정치 조작’ 영풍제련소 임원, 징역 1년 2개월 실형",http://www.newsmin.co.kr/news/42853/,대기오염물질 측정치를 조작해 구속기소 된 영풍 석포제련소 임원이 징역 1년 2개월 ...,대기오염,대기오염물질 측정치를 조작해 구속기소 된 영풍 석포제련소 임원이 징역 1년 2개월 ...,"[대기, 오염, 물질, 측정치, 조작, 구속, 기소, 영풍, 석포, 제련소, 임원,...","{'대기': 8, '오염': 7, '물질': 5, '측정치': 1, '조작': 2,...",202
...,...,...,...,...,...,...,...,...,...,...
250,343144,한국가스공사,'LNG 벙커링'으로 친환경·수익 동시만족,https://www.fnnews.com/news/202104201658176648,진행중 청년들 표심 잡으려 '군 가산점' 부활? fnSURVEY\n\n가스公 새 비...,대기오염,진행중 청년들 표심 잡으려 군 가산점 부활 fnsurvey가스 새 비즈니스모델 창출...,"[진행, 중, 청년, 표, 가산점, 부활, 가스, 비즈니스, 모델, 창출, 경유, ...","{'진행': 1, '중': 1, '청년': 1, '표': 1, '가산점': 1, '...",274
251,343437,대상,"충북지속가능발전협, 초·중·고 대상 미세먼지 대응 교육 추진",http://www.jbnews.com/news/articleView.html?id...,11월까지 시행… 미세먼지 대응교육 강사 양성해 투입\n\n안성수 기자 seongs...,대기오염,11월까지 시행 미세먼지 대응교육 강사 양성해 투입 중부매일 무단전재 및 수집 재배...,"[월, 시행, 미세먼지, 대응, 교육, 강사, 양성, 투입, 중부, 무단, 전재, ...","{'월': 3, '시행': 1, '미세먼지': 10, '대응': 5, '교육': 1...",182
252,344252,SK가스,용광로 배출가스 불투명도로 측정 관리한다,https://www.seoul.co.kr/news/newsView.php?id=2...,"환경부, 용광로 배출 오염물질 관리기준 신설\n\n밸브 개방시 20분간 불투명도 2...",대기오염,환경부 용광로 배출 오염물질 관리기준 신설밸브 개방시 20분간 불투명도 20 초과 ...,"[환경부, 용광, 배출, 오염, 물질, 관리, 기준, 신설, 밸브, 개방, 시, 분...","{'환경부': 8, '용광': 8, '배출': 9, '오염': 6, '물질': 7,...",270
253,344452,현대제철,제철소 브리더밸브 오염물질 논란 2년 만에 일단락 - 서울경제,https://www.sedaily.com/NewsVIew/22L6LUIT59,viewer 현대제철소 당진공장 내부 모습. /서울경제 DB\n\n< 저작권자 ⓒ ...,대기오염,현대제철소 당진공장 내부 모습. 서울경제 db지난 2019년 환경단체의 고발로 불...,"[현대, 제철소, 당진, 공장, 내부, 모습, 서울, 경제, 년, 환경, 단체, 고...","{'현대': 1, '제철소': 6, '당진': 2, '공장': 1, '내부': 1,...",161


### 길이가 2 이상인 명사들만 추출

In [12]:
df_air_pol['contents_nouns_len>1'] = df_air_pol['contents_nouns'].apply(lambda x: [individual_word for individual_word in x if len(individual_word) > 1])

In [13]:
df_air_pol['nouns_no_len>1'] = df_air_pol['contents_nouns_len>1'].apply(lambda x: len(x))

In [14]:
df_air_pol

,id,keyword,title,link,contents,y,contents_clean,contents_nouns,nouns_counter,nouns_no,contents_nouns_len>1,nouns_no_len>1
0,11129,삼성전기,"내연기관 자동차 시대 끝?…“디젤차 사, 말아?”",http://www.enewstoday.co.kr/news/articleView.h...,"[이뉴스투데이 윤진웅 기자]\n\n#서울에 사는 직장인 박수남(가명, 41세) 씨는...",대기오염,상황이 이렇자 자동차 회사들 역시 국내와 수입을 막론하고 미래차 개발에 박차를 가하...,"[상황, 자동차, 회사, 국내, 수입, 미래, 차, 개발, 박차, 국내, 자동차, ...","{'상황': 1, '자동차': 5, '회사': 1, '국내': 2, '수입': 2,...",289,"[상황, 자동차, 회사, 국내, 수입, 미래, 개발, 박차, 국내, 자동차, 시장,...",225
1,102294,GS건설,허태수 GS 회장의 '친환경 경영'…공허하게 들리는 이유는,http://isplus.joins.com/article/933/23810933.html,GS그룹의 새 수장인 허태수 회장은 그룹의 미래 설계를 그리며 '친환경 경영'을 강...,대기오염,gs그룹의 새 수장인 허태수 회장은 그룹의 미래 설계를 그리며 친환경 경영을 강조하...,"[그룹, 수장, 허태수, 회장, 그룹, 미래, 설계, 친환경, 경영, 강조, 포스트...","{'그룹': 2, '수장': 1, '허태수': 1, '회장': 6, '미래': 1,...",338,"[그룹, 수장, 허태수, 회장, 그룹, 미래, 설계, 친환경, 경영, 강조, 포스트...",287
2,106248,한국금융지주,"한국전력공사, '인도네시아 화력발전소 건설 사업' 이사회서 의결 ...",http://www.econonews.co.kr/news/articleView.ht...,[이코노뉴스=조승환 기자] 한국전력공사가 '인도네시아 자바 9·10호기 화력발전소 ...,대기오염,한국전력공사가 인도네시아 자바 910호기 화력발전소 건설 사업 추진을 일단 멈추기로...,"[한국전력공사, 인도네시아, 자바, 호기, 화력, 발전소, 건설, 사업, 추진, 한...","{'한국전력공사': 2, '인도네시아': 2, '자바': 1, '호기': 1, '화...",57,"[한국전력공사, 인도네시아, 자바, 호기, 화력, 발전소, 건설, 사업, 추진, 한...",50
3,150481,현대중공업지주,비산먼지 저감 살수차 운영,https://www.kbmaeil.com/news/articleView.html?...,"포스코와 현대제철(주), 동국제강(주), 에코시스템(주), (주)제철세라믹, (주)...",대기오염,포스코와 현대제철 동국제강 에코시스템 제철세라믹 세기 등 6개 회사들이 오는 31일...,"[포스코, 현대제철, 동국제강, 에코, 시스템, 제철, 세라믹, 세기, 등, 개, ...","{'포스코': 1, '현대제철': 1, '동국제강': 1, '에코': 1, '시스템...",36,"[포스코, 현대제철, 동국제강, 에코, 시스템, 제철, 세라믹, 세기, 회사, 폭염...",29
4,155072,오뚜기,"‘대기오염 측정치 조작’ 영풍제련소 임원, 징역 1년 2개월 실형",http://www.newsmin.co.kr/news/42853/,대기오염물질 측정치를 조작해 구속기소 된 영풍 석포제련소 임원이 징역 1년 2개월 ...,대기오염,대기오염물질 측정치를 조작해 구속기소 된 영풍 석포제련소 임원이 징역 1년 2개월 ...,"[대기, 오염, 물질, 측정치, 조작, 구속, 기소, 영풍, 석포, 제련소, 임원,...","{'대기': 8, '오염': 7, '물질': 5, '측정치': 1, '조작': 2,...",202,"[대기, 오염, 물질, 측정치, 조작, 구속, 기소, 영풍, 석포, 제련소, 임원,...",178
...,...,...,...,...,...,...,...,...,...,...,...,...
250,343144,한국가스공사,'LNG 벙커링'으로 친환경·수익 동시만족,https://www.fnnews.com/news/202104201658176648,진행중 청년들 표심 잡으려 '군 가산점' 부활? fnSURVEY\n\n가스公 새 비...,대기오염,진행중 청년들 표심 잡으려 군 가산점 부활 fnsurvey가스 새 비즈니스모델 창출...,"[진행, 중, 청년, 표, 가산점, 부활, 가스, 비즈니스, 모델, 창출, 경유, ...","{'진행': 1, '중': 1, '청년': 1, '표': 1, '가산점': 1, '...",274,"[진행, 청년, 가산점, 부활, 가스, 비즈니스, 모델, 창출, 경유, 선박, 친환...",227
251,343437,대상,"충북지속가능발전협, 초·중·고 대상 미세먼지 대응 교육 추진",http://www.jbnews.com/news/articleView.html?id...,11월까지 시행… 미세먼지 대응교육 강사 양성해 투입\n\n안성수 기자 seongs...,대기오염,11월까지 시행 미세먼지 대응교육 강사 양성해 투입 중부매일 무단전재 및 수집 재배...,"[월, 시행, 미세먼지, 대응, 교육, 강사, 양성, 투입, 중부, 무단, 전재, ...","{'월': 3, '시행': 1, '미세먼지': 10, '대응': 5, '교육': 1...",182,"[시행, 미세먼지, 대응, 교육, 강사, 양성, 투입, 중부, 무단, 전재, 수집,...",158
252,344252,SK가스,용광로 배출가스 불투명도로 측정 관리한다,https://www.seoul.co.kr/news/newsView.php?id=2...,"환경부, 용광로 배출 오염물질 관리기준 신설\n\n밸브 개방시 20분간 불투명도 2...",대기오염,환경부 용광로 배출 오염물질 관리기준 신설밸브 개방시 20분간 불투명도 20 초과 ...,"[환경부, 용광, 배출, 오염, 물질, 관리, 기준, 신설, 밸브, 개방, 시, 분...","{'환경부': 8, '용광': 8, '배출': 9, '오염': 6, '물질': 7,...",270,"[환경부, 용광, 배출, 오염, 물질, 관리, 기준, 신설, 밸브, 개방, 투명, ...",227
253,344452,현대제철,제철소 브리더밸브 오염물질 논란 2년 만에 일단락 - 서울경제,https://www.sedaily.com/NewsVIew/22L6LUIT59,viewer 현대제철소 당진공장 내부 모습. /서울경제 DB\n\n< 저작권자 ⓒ ...,대기오염,현대제철소 당진공장 내부 모습. 서울경제 db지난 2019년 환경단체의 고발로 불...,"[현대, 제철소, 당진, 공장, 내부, 모습, 서울, 경제, 년, 환경, 단체, 고...","{'현대': 1, '제철소': 6, '당진': 2, '공장': 1, '내부': 1,...",161,"[현대, 제철소, 당진, 공장, 내부, 모습, 서울, 경제, 환경, 단체, 고발, ...",136


## Sample들 확인

### 뉴스 본문으로 확인

In [15]:
df_air_pol['contents_clean'][0]

'상황이 이렇자 자동차 회사들 역시 국내와 수입을 막론하고 미래차 개발에 박차를 가하고 있다.먼저 국내 자동차 시장에서 1위를 달리고 있는 현대자동차는 내년부터 순수 전기차를 본격적으로 생산하겠다는 계획을 밝혔다. 포니를 모티브로 만들어진 순수 전기콘셉트카 45 ev가 suv 스타일로 양산할 계획이다. 아울러 전기차 전용 플랫폼인 egmp 생산 라인을 확보하기 위해 울산 1공장과 2공장의 내연기관차 생산라인 일부를 전기차 전용라인으로 교체 중이다. 현대차 노조에 따르면 내년부터 전기차 전용 라인에서 45 ev 양산차 등 순수 전기차 7만 4000여 대 2022년에는 8만 9000여 대를 생산할 전망이다.수입 자동차 판매 부문 1위를 유지하고 있는 메르세데스벤츠는 2039년까지 탄소 중립 달성을 목표로 미래차에 역량을 집중하고 있다. 전기차 더뉴eqc에 이어 최근 비전eqs를 공개하며 미래차 시장을 이끌어갈 전략을 발표하기도 했다. 또한 성공적인 미래차 시장 선점을 위해 전시공간까지 미래지향적으로 바꿔나가며 보조를 맞추고 있다. 비전eqs는 내년 양산될 예정이다.전국적으로 전기차 충전소도 급증하고 있다. 환경부 조사 결과에 따르면 전국 전기차 충전소는 총 2만415개로 집계됐다. 1년 전과 비교해 두 배 이상 늘어난 수치다. 충전소가 가장 많은 곳은 경기도로 총 3615개를 보유했다. 다음으로는 제주가 2821개로 두 번째로 전기차 충전소가 많았다. 서울 지역 내 충전소는 1980개로 파악됐다.이처럼 미래차에 대한 개발과 기반시설 준비가 전반으로 확대되면서 업계 관계자들은 전기차 시대가 예상보다 더 빨라질 것으로 전망하고 있다. 또한 학계 역시도 전기차에 대한 기대감이 큰 것으로 나타났다.김필수 대림대학교 자동차학과 교수는 선진국을 100라고 했을 때 우리나라는 85 정도 수준이라고 볼 수 있는데 15는 23년 정도 차이라며 자율주행 관련 라이다센서가 아직 국산화가 안 돼 격차가 있는 것이라고 말했다.그러면서 각개전투식으로 보면 기술력이 뛰어나지만 융합적인 기술

**키워드: 전기차, 탄소, 중립, 배터리, 리튬, 충전소, 내연기관**

In [16]:
df_air_pol['contents_clean'][1]

'gs그룹의 새 수장인 허태수 회장은 그룹의 미래 설계를 그리며 친환경 경영을 강조하고 있다. 포스트 코로나 시대를 대비해 계열사들의 발 빠른 대응책 마련을 촉구하고 있지만 친환경 경영과는 거리가 먼 사고가 잇따르며 엇박자가 나고 있다.허 회장은 지난 17일 서울 gs남촌리더십 센터에서 열린 gs 임원 포럼에서 최근 글로벌 공급망이 재편되고 모바일 활성화 및 비대면 경제의 확산을 디지털 역량 강화 등 우리의 부족한 점을 고도화하는 계기로 삼고 향후 포스트 코로나 시대를 대비하자고 당부했다.이번 포럼은 지난 1월 허 회장이 취임한 이후 두 번째 확대 경영 회의다. 코로나19의 팬데믹 이후 gs의 사업전략을 점검하고 위기 극복을 위한 방안 마련을 위해 ceo들이 머리를 맞댔다. 전임 허창수 회장이 용퇴하면서 남긴 디지털 혁신을 이끌 새 리더로 낙점된 허 회장은 디지털 전환 작업에 속도를 내고 있다.디지털 전환과 함께 허 회장이 강조한 핵심 키워드가 바로 친환경 경영이다. 이번 포럼에서도 향후 친환경을 통한 지속 가능 경영 실천의 중요성이 크게 부각될 것이라며 이제 우리의 내부 역량을 이런 외부의 변화에 맞춰 혁신해야 한다고 역설했다.gs의 핵심 계열사인 gs칼텍스와 gs홈쇼핑 등이 친환경 경영 실천에 나서고 있다. gs칼텍스는 친환경 에너지 화학 기업 비전을 내세우며 지난 4월 여수공장 생산시설을 가동하는 연료 전량을 액화천연가스로 대체했다. 기존 저유황 중유보다 lng의 이산화탄소 배출량은 74로 줄어드는 것으로 알려졌다.gs홈쇼핑도 지난 12일 컬러잉크를 사용하지 않는 친환경 배송 박스 도입을 발표했다. 지난해부터 친환경 포장재를 확대하는 추세다. 화학성분이 포함된 아이스팩 대신 얼린 생수를 냉매로 활용하는 친환경 배송을 도입하기도 했다.하지만 친환경 경영 방침은 공허한 외침이 되고 있다. gs칼텍스의 폐수 유출 사고가 끊이지 않고 있고 대기오염물질 배출량 조작 사건 이후로도 재발 방지 노력이 미흡하다는 평가다.전라남도 동부지역본부는 지난 4월 8일 여수산단의

**키워드: 이산화탄소 배출, 대기오염물질, 액화천연가스, 저유황, 대기** 

In [17]:
df_air_pol['contents_clean'][2]

'한국전력공사가 인도네시아 자바 910호기 화력발전소 건설 사업 추진을 일단 멈추기로 했다.한국전력공사는 26일 서울 서초구 한전아트센터에서 이사회를 열고 해당 안건에 대한 의결을 보류했다.이 사업은 인도네시아 정부가 지정한 국가 전력 인프라 사업으로 총사업비만 약 34억 달러에 달한다. 이를 통해 자카르타 인근에 1gw 규모 석탄화력발전소 2기를 짓게 된다.이번 결정은 정치권과 환경단체 등에서 불거진 수익성 논란과 온실가스 생산으로 인한 환경 오염 등에 대한 부정적인 여론을 반영한 것으로 풀이된다.'

**키워드: 석탄, 화력, 온실가스**

In [18]:
df_air_pol['contents_clean'][3]

'포스코와 현대제철 동국제강 에코시스템 제철세라믹 세기 등 6개 회사들이 오는 31일까지 폭염 및 비산먼지 줄이기 대책의 일환으로 철강관리공단 내 주요 도로에 물 뿌리기 작업을 벌인다. 23일 공단업체에서 지원된 살수차가 도로에 물을 뿌리고 있다. 포항시제공 경북매일 '

**키워드: 먼지**

In [19]:
df_air_pol['contents_clean'][4]

'대기오염물질 측정치를 조작해 구속기소 된 영풍 석포제련소 임원이 징역 1년 2개월 실형을 선고받았다.5일 대구지방법원 서부지원 제5형사단독은 영풍제련소 이사 한 모 씨에게 위계공무집행방해 등의 혐의로 징역 1년 2개월 대기오염 측정대행업체인 현대공해측정주식회사 사내이사 박 모 씨에게 징역 10월 실형을 선고했다. 삼안환경화학측정 현대공해측정주식회사 다른 관계자 4명은 집행유예가 떨어졌다.재판부에 따르면 한 씨와 박 씨는 영풍제련소 이미지 재고를 위해 공모해 대기오염물질 측정 분석 결과를 조작했다. 재판부는 이들이 환경부의 점검에 대비해 업무용 pc의 하드디스크 교체 담당자 휴대전화 교체 등 증거를 인멸한 것으로 보고 있다.재판부는 영풍제련소의 환경 업무를 총괄하는 한 이사가 과거에도 토양환경보전법 대기환경보전법 등을 위반해 6차례 벌금형 처벌 전력이 있는 점을 실형 선고에 반영했다.재판부는 기본부과금 면제와 영풍제련소가 대기오염물질을 적절히 관리한다는 대외적 이미지 재고를 위해 공모해 대기오염물질 측정 결과를 거짓으로 기록해 담당 공무원의 부과금 집무 집행을 방해했다며 범행이 장기간 조직적 계획적으로 이뤄져 법 위반 정도가 중하다고 지적했다.이어 법률 준수나 개선책 마련에 노력하지 않고 수치 자료를 삭제했다며 대기오염물질 배출에 관한 측정제도 신뢰성을 저하하고 대기질 악화와 환경 훼손 국민 생명권 위해 가능성이 있다고 덧붙였다.측정 대행업체는 재판 과정에서 영풍제련소와 갑을관계 때문에 위법 행위가 불가피했다고 주장했지만 재판부는 환경오염방지에 역할과 의무를 다하지 못했다고 꼬집었다.'

**키워드: 대기오염 물질**

In [20]:
df_air_pol['contents_clean'][5]

'봉화 영풍석포제련소에 이어 포스코 포항제철소도 조업정지 행정처분을 받을 상황에 놓였다. 1973년 포항제철소가 영업허가를 받은 이후 수십년간 대기오염물질을 ...'

**키워드: 대기오염 물질**

In [21]:
df_air_pol['contents_clean'][6]

'현대제철 당진제철소가 대기오염물질 방지 시설 없이 대기 중에 오염물질을 내뿜은 것으로 드러났다.또 현대제철은 배출 시설을 신고하지 않은 채 고로를 운영하거나 고장 난 시설을 방치하기도 했다. 이에 따라 충남도는 이 제철소에 대해 조업정지 10일과 사용 중지 처분을 내리고 검찰에 고발하기로 했다.7일 충남도에 따르면 도 당진시 경기도 환경단체 관계자 등 30명은 지난 2일 현대제철 당진제철소 등 도내 사업장에 대한 특별 합동 점검을 벌였다. 점검 결과 현대제철 당진제철소가 제2고로 용광로 정비 과정에서 발생한 먼지를 방지시설을 설치하지 않고 주로 새벽 시간대 대기 중에 곧바로 내보낸 것으로 드러났다.또 연주 공정에서 기타로를 설치운영하면서 대기오염물질 배출 신고를 하지 않았던 것으로 확인됐다.'

**키워드: 대기오염 물질, 오염 물질, 제철소**

In [22]:
df_air_pol['contents_clean'][7]

'lg전자가 퓨리케어 대형 공기청정기를 설치한 상업시설과 공공시설에 미세먼지 관리지역을 알리는 스티커를 부착해 제품 홍보에 나선다고 18일 밝혔다.lg전자 제공이미 제품을 구입한 경우에도 스티커 부착을 신청할 수 있다. 스티커에는 지금 계신 매장은 미세먼지 관리지역입니다라고 써 있다. 제품에도 깨끗한 공기가 나오는 중입니다라는 안내 문구가 있다.lg전자 는 lg 퓨리케어 대형 공기청정기 구매 고객을 조사한 결과 40 이상이 업소 등 상업용도의 구매였다고 밝혔다. 구매 이유로는 응답 고객의 56가 넓은 청정면적을 구매 후 가장 만족하는 점으로는 41가 빠른 청정성능을 꼽았다.lg 퓨리케어 대형 공기청정기의 청정면적은 158제곱미터로 한국공기청정협회로부터 ca 인증을 받은 제품 중 가장 넓어 가정용 공기청정기로는 다소 부족한 상업용 공간에 적합하다고 lg전자 측은 설명했다.공기 흡입구와 토출구가 제품의 앞쪽에 있어서 제품 뒷면이나 옆면에 별도 공간을 둘 필요 없이 벽에 밀착해 설치할 수 있기 때문에 공간 활용도도 높다.6단계 토탈 케어 플러스 필터 시스템은 미세먼지와 초미세먼지 이산화황 이산화질소 톨루엔 포름알데이드 같 은 유해물질을 잡아낸다. 한국천식알레르기협회와 영국알레르기협회로부터 알레르기 질환 예방과 환경관리에 적합한 제품으로 인증도 획득했다.백승태 lg전자 rac사업담당 상무는 청정면적과 청정성능을 모두 향상시킨 lg 퓨리케어 대형 공기청정기를 설치한 매장이 깨끗한 실내 공기로 차별화된 경쟁력을 확보할 수 있도록 지속적으로 지원하겠다고 말했다'

**키워드: 공기, 청정기, 미세먼지, 초미세먼지, 이산화황, 이산화질소, 톨루엔, 포름알데이드**

In [23]:
df_air_pol['contents_clean'][8]

'사회종합 대한유화 온산공장 강력 처벌환경오염에 경고 메시지 공장장 실형벌금 최고액 선고울산지법 차후범행 예방 차원 승인 2018.07.04 214437 지난해 6월 공장 굴뚝에서 화염과 매연을 배출한 대한유화 온산공장의 공장장이 4일 법정 구속됐고 해당 업체에는 법이 정하는 최고액의 벌금이 선고됐다. 기업 이익을 위해 환경 오염을 대수롭지 않게 여긴 책임자와 타 기업들에게 강한 경고의 메시지를 보내기 위해 실형을 선고했다는 것이 재판부의 설명이다.울산지법은 대기환경보전법 위반 혐의로 기소된 대한유화 온산공장장 a씨에게 징역 4월을 선고하고 법정구속했다. 또 대한유화에는 법정 상한선인 벌금 1500만원을 선고했다.a씨와 대한유화는 지난해 6월9일 오후 12시38분께부터 1시17분께까지 온산공장 사업장에서 플레어스택의 처리 용량을 초과하는 에틸렌과 프로필렌 벤젠 등의 탄화수소류를 플레어스택에 유입하는 등 총 8차례에 걸쳐 매연을 발생시킨 혐의로 재판에 넘겨졌다.a씨 등은 사건의 형벌조항인 대기환경보전법 제91조 제2의5호가 반성적 고려에서 폐지돼 면소돼야 한다고 주장했다. 또 플레어스택은 비산배출시설이 아니고 대기오염방지시설로 법의 관리를 받는 만큼 이를 비산배출시설로 봐 규제하는 것은 이중처벌이라며 무죄를 주장했다.재판부는 면소 주장에 대해 개정된 법의 시행일은 2018년 11월29일이기 때문에 현재 해당 법이 폐지됐다고 볼만한 근거가 없다는 등의 이유로 받아들이지 않았고 무죄 주장에 대해서도 플레어스택은 비산배출시설이 맞고 다른 행정규제를 받아 이중처벌이라는 주장 역시 독자적 견해에 불과하다고 유죄를 인정했다.재판부는 피고인들은 매우 중한 범죄를 저지르고도 겉으로는 반성하는 듯 하지만 이는 처벌조항이 삭제된 개정법의 시행일인 오는 11월29일까지 재판을 지연시키기 위한 수단으로밖에 느껴지지 않는다고 지적했다.또 피고인 및 변호인의 주장대로 재판 속도를 늦추면 11월29일 이후 면소로 끝나게 될 것이라며 집행유예 이하의 형을 선고한다면 피고인은 재판 출석 외에

**키워드: 화염, 매연, 에틸렌,프로필렌, 벤젠, 에틸렌과 프로필렌 벤젠, 대기오염, 비산, 배출**

In [24]:
df_air_pol['contents_clean'][9]

'봉화 경북 봉화군 소재 영풍 석포제련소는 지난 3일 대구의 한 신문을 통해 보도된 정수근 영풍제련소 환경오염 및 주민건강 피해 공동대책위원장의 허위사실 유포에 대해 법적으로 대응하겠다고 밝혔다.영풍은 5일 보도자료를 통해 정 위원장은 석포제련소가 낮에는 조업량을 줄이고 밤에 대폭 늘리는 꼼수를 부려 밤마다 유해한 가스가 석포 하늘을 뒤덮는다는 괴담을 주장했다며 이는 전혀 근거 없는 주장일 뿐만 아니라 현재 공장 시설과 제련 공정에서는 기술적으로 불가능한 허위사실을 대대적으로 유포했다고 했다.또한 석포제련소가 한밤중에 유해가스를 의도적으로 대량 배출하는 범죄행위를 한다는 주장을 확실한 근거도 없이 카더라 통신을 인용해 아니면 말고 식으로 한다는 것은 있을 수 없다며 이는 회사와 그 종사자들의 신뢰도와 명예를 심각하게 침해한 것으로 그에 대한 응분의 책임을 져야 할 것이다고 덧붙였다.아울러 정 위원장은 이따이이따이병 등 100년 전 일본 광산에서 발생한 환경사고를 이와 전혀 무관한 영풍에 덮어씌우는 폭거를 지속적으로 저지르고 있다고 설명했다.특히 영풍 tms는 굴뚝이나 방류구의 배출물질을 24시간 측정해 실시간으로 환경공단으로 전송하는 장비이다며 tms는 관련 법령에 따라 그 설치 대상과 측정 항목 등이 정해지고 그 관리 역시 법령에 따라 엄격하게 이뤄지는 것으로 영풍이 임의로 설치 여부를 결정할 수가 없다고 강조했다.또 정 위원장은 영풍 측이 의도적으로 tms를 설치하지 않는 것처럼 주장할 뿐만 아니라 설치되지 않은 굴뚝으로 대량의 오염물질을 불법적으로 배출한다는 사실과 전혀 다른 근거 없는 주장을 하고 있다고 덧붙였다.이어 거대한 일관 공정인 제련 공정의 특성상 낮에는 조업량을 줄였다가 밤에는 갑자기 조업량을 늘리는 식의 공장 가동은 제련업에서 기술적으로 불가능한 것이라며 단일 기업에 대한 대응을 넘어 제련 산업을 절단낼 수 있는 온갖 언어적 폭력을 서슴지 않는 작태는 더 이상 참지 않겠다고 밝혔다.한편 영풍제련소 환경오염 및 주민건강 피해 공동대책위원회는지

**키워드: 유해, 가스, 배출, 오염**

In [25]:
df_air_pol['contents_clean'][10]

'지역 시민단체 집단에너지사업법 위반 혐의로 고발 예정gs포천그린 고발 여부 확인 안 돼 아직 대응 계획 없어포천석탄발전소 반대 공동 투쟁본부투데이신문 경기도 포천시 주민들과 시민단체가 gs포천그린에너지를 집단에너지사업법 위반 혐의로 고발할 계획이다. gs포천그린에너지는 지난해 8월부터 석탄화력발전소의 상업가동을 시작했지만 산업단지 굴뚝 일원화 등 환경영향평가 협의 내용을 이행하지 않아 대기오염을 우려하는 주민들의 반발에 부딪혔다.10일 포천석탄발전소 반대 공동 투쟁본부에 따르면 석투본은 오는 15일 미비된 서류를 보완해 지역단체들과 공동으로 gs포천그린에너지를 고발할 방침이다. 고발 준비에는 석탄발전소반대공동투쟁위원회 포천시주민자치연합회 한국기후환경포천네트워크 포천시이통장연합회 등 32개 단체가 동참했다.포천석탄화력발전소는 지난 2015년 11월 산업통상자원부로부터 공사계획 승인을 받았고 같은해 12월 말 착공에 들어갔다. 작년 8월부터는 본격적인 상업 가동에 돌입했다. 이 사업은 경기도 포천시 신북면 신평3리의 무허가시설과 신평2리 염색집단화단지의 개별 보일러를 일원화 해 지역 환경을 개선하고자 추진됐다.현재 신평3리는 장자산업단지가 조성되며 자연스럽게 문제가 해결이 됐지만 신평2리는 아직 고형폐기물연료나 벙커c유 보일러들이 가동되고 있는 상황이다. 지역 주민들과 시민단체들은 이에 따른 대기오염 우려하고 있으며 상업 가동 자체가 환경영향평가 협의 내용인 신평23리 굴뚝 일원화를 이행하지 않은 불법 행위라고 지적하고 있다. 이에 따라 지난 9일에는 이들 단체들이 모여 gs포천그린에너지의 불법 행위를 주회견을 열기도 했다.이와 관련 석투본 홍영식 사무국장은 집단어네지사업법 위반으로 고발을 계획하는 것이라며 환경영향평가법에 따라 신평2리에도 굴뚝일원화를 완료해 증기를 모두 공급해야 하지만 현재 이를 이행하지 않고 발전소를 100 가동하고 있다고 꼬집었다.그는 이어 집단에너지사업법에서는 20 이상 에너지 사용에 변경이 있을 경우 산업통상자원부 장관에게 변경허가를 

**키워드: 석탄, 발전소, 굴뚝, 대기오염**

In [26]:
df_air_pol['contents_clean'][11]

'포스코 건설 배짱 공사하다 딱 걸려 경찰에 고발당해 2020.09.20 1408  2020.09.20 140817일 포스코 건설이 공사하는 포스코 포항홍보관 및 문화조성공사를 하면서 세륜.세차 시설 2대를 설치하지 않고 공사를 하다 포항시로부터 경찰에 고발당했다.포항포스코 포항홍보관 및 문화조성공사 현장 환경보전법 위반포스코 건설이 배짱으로 공사를 하다 경찰에 고발 당하는 수모를 겪었다.포항시는 지난17일 포스코 건설을 대기환경보전법 비산먼지억제조치 미이행 위반혐의로 포항남부경찰서에 고발했다고 밝혔다.시는 이보다 앞서 15일 포스코 건설이 공사하는 포스코 포항홍보관 및 문화조성공사를 하면서 세륜.세차 시설 2대를 설치하지 않고 공사를 하다 적발했다.또 포스코 건설이 이 공사장에 살수 시설을 설치하겠다고 변경 신고를 하고도 이를 이행하지 않고 공사를 하다 적발 된 사후조치다.한편 포스코 건설은 포항시 남구 괴동동 일원에 연면적 3492.29건축면적 2052.04 총사업부지 4만34 지하1층 지상2층 규모의 포스코 포항홍보관 및 문화조성공사를 내년 2월28일까지 준공 하게 된다.tktf.co.kr'

**키워드: 비산, 먼지**

In [27]:
df_air_pol['contents_clean'][12]

'롯데케미칼측은 해당 소각시설에서 허용기준을 초과한 고농도의 암모니아가 배출될 가능성이 없고 오류가 개입됐을 가능성을 배제할 수 없다고 주장했지만 법원은 ...'

**키워드: 암모니아, 배출**

In [28]:
df_air_pol['contents_clean'][13]

'한국중부발전5600억 들여 최신 환경설비 개선석탄발전소 오염물질 배출 68한국중부발전이 국민 건강을 지키기 위한 미세먼지 감축에 총력을 기울이고 있다. 지역사회와 긴밀하게 협력해 발전소의 오염물질 방출량을 획기적으로 저감하는 동시에 취약계층 지원을 위해 적극적으로 노력하고 있다.한국중부발전은 미세먼지를 포함한 대기오염물질 배출량을 저감하기 위해 최신 환경설비 설치 및 개선에 지난해까지 약 5600여억원을 투자했다. 그 결실로 석탄발전소의 대기오염물질 배출량이 2015년 대비 68에 상당하는 약 2만4000t 줄었다.지난해 기준 저유황탄 비중을 85까지 확대하고 보령 7호기 및 신보령 1호기 탈황설비 용량을 늘리는 등의 성능 보강 노력이 주효했다. 특히 보령 3호기는 세계 최고 수준의 설비를 도입해 황산화물 5ppm 질소산화물 7ppm 먼지 2mgsm등 국내는 물론 해외에서도 최고 수준의 저감 성능을 입증했다.한국중부발전은 저탄장 비산먼지 발생을 차단하기 위해 오는 2024년까지 약 2600억원을 투자해 모든 저탄장의 옥내화를 추진한다. 유연탄 수송 선박에서 하역할 때 필요한 전력도 발전소 생산 전력으로 대체함으로써 하역 과정의 선박 대기오염물질도 약 98 감축할 계획이다. 이를 위해 올 하반기까지 육상전력공급장치를 구축한다.또한 기존 석탄발전소의 대기오염물질 감축 로드맵을 보완해 석탄 lng 및 유류 발전을 모두 포함하는 대기오염물질 감축 로드맵을 재수립했다. 지난해 말 발표된 정부의 미세먼지 종합대책 및 석탄발전의 lng 대체 건설 등을 반영해 석탄발전의 환경설비 성능 개선 및 보강 계획을 보완했다. 재수립된 대기오염물질 감축 로드맵에 따르면 석탄 lng 및 유류 발전 등 한국중부발전이 운영 및 건설 중인 모든 발전기에 대해 2025년까지 대기오염물질 배출량을 2015년 대비 82 감축한다. 또 2025년까지 추가적으로 약 8000억원을 투자할 계획이다.한국중부발전은 발전소뿐만 아니라 대기오염물질 저감 노력을 지역사회 전반에 확산하는 데도 정성을 쏟고 있다

**키워드: 석탄, 발전소, 오염물질, 배출, 미세먼지, 대기 오염, 배출량, 탈황, 청정, 연료, 공기, 청정**

In [29]:
df_air_pol['contents_clean'][14]

'올 상반기 대기오염물질을 초과 배출한 충남 당진의 한 사업장 전경. 사진대전일보 db현대제철 현대오일뱅크 등 충남지역 주요 사업장들이 올 상반기에도 대기오염물질을 초과 배출한 것으로 나타났다.충남도는 도내 굴뚝자동측정기기 부착사업장 63개소 중 29개소에 대해 2020년 상반기 초과배출부과금 23억 7300만 원을 부과했다고 22일 밝혔다. 이들 대기오염물질 초과배출 사업장 29개 가운데 18개 사업장이 천안 아산 당진 서산 등 충남 서북부에 위치하고 있는 것으로 확인됐다.이를 지역별로 보면 서산이 현대오일뱅크 현대케미칼 현대코스모 등 6곳으로 가장 많으며 이어 천안이 영흥산업환경 삼광글라스천안유리공장 삼정펄프천안공장 등 5곳 당진이 현대제철 지에스이피에스 등 4곳으로 나타났다.대기 초과배출부과금은 대기오염물질 배출허용기준을 초과 배출하는 사업장에 대해 부과금을 정하는 제도로 배출항목 중 질소산화물 황산화물 염화수소 먼지의 배출량 등을 산정해 부과한다.굴뚝자동측정기기 부착 사업장을 대상으로 하는 초과배출부과금은 상하반기 진행하며 지난해에는 상하반기 총 49억 5800만 원을 부과했다. 초과배출부과금은 환경오염을 저감시키기 위한 환경오염방지기금의 재원으로 사용한다.충남도 관계자는 굴뚝자동측정기기 부착사업장은 사업장 대기오염물질 총량관리제 시행으로 인해 점차 늘어날 전망이라며 도민의 건강과 안전을 위해 대기환경오염을 줄이는 다양한 정책을 발굴할 것이라고 말했다.'

**키워드: 대기, 오염, 물질, 굴뚝, 배출**

In [30]:
df_air_pol['contents_clean'][15]

'롯데케미칼이 전라남도를 상대로 대기오염물질 초과 배출 부과금 부과처분 취소 소송을 제기했으나 패소했다. 전라남도가 대기오염물질을 초과 배출한 혐의로 롯데케미칼에 부과금을 부과한 것은 정당하다는 게 법원의 판단이다.전남도 보건환경연구원은 지난해 5월 롯데케미칼 여수 1공장 폐가스 소각시설에서 암모니아가 배출허용기준을 초과해 배출했다는 혐의로 개선명령과 함께 초과배출부과금 4천여만원을 부과했다.전남도 보건환경연구원은 앞서 롯데케미칼 여수 1공장 폐가스 소각시설에서 시료를 채취해 대기오염물질 배출원 및 배출량 측정조사를 실시한 시험성적서를 근거해 행정처분을 내렸다.롯데케미칼측은 전남도 보건환경연구원측이 측정조사 과정에서 공정시험기준에 따른 암모니아 농도 측정법을 사용하지 않는 등 공정시험기준의 절차를 위반해 결과를 신뢰할 수 없다며 소송을 냈다22일 광주지역 언론사와 법조계에 따르면 광주지법 행정 2부는 롯데케미칼이 전남도지사를 상대로 제기한 대기초과 배출부과금 부과처분 취소 소송에서 해당 소각시설에서 암모니아가 초과 배출됐다며 원고 패소 판결했다.법원은 해당 소각시설에서 허용기준을 초과한 고농도의 암모니아가 배출될 가능성이 없고 오류가 개입됐을 가능성을 배제할 수 없다는 롯데케미칼측 주장에 대해 이유없다고 기각했다.재판부는 또 공정시험기준에 따른 암모니아 농도 측정법을 사용하지 않았더라도 해당 사건 측정조사가 잘못됐다고 보기 어렵다고 판시했다.재판부는 이어 2017년 기준 해당 회사 여수 1공장에서 대기로 배출되는 암모니아양이 2778로 등록된 점 2018년 배출되는 오염물질에 암모니아가 포함돼 있다는 취지의 배출시설 변경 신고를 한 점 등을 감안하면 해당 소각시설에서 암모니아가 배출될 가능성이 없다는 주장을 받아들이기 어렵다고 밝혔다.'

**키워드: 대기, 가스, 소각, 암모니아**

### 명사로만 확인

In [31]:
print(df_air_pol['nouns_counter'][0])

Counter({'차': 14, '전기차': 11, '미래': 10, '자동차': 5, '전기': 5, '충전': 5, '수': 5, '생산': 4, '라인': 4, '년': 4, '개': 4, '것': 4, '교수': 4, '시장': 3, '내년': 3, '순수': 3, '양산': 3, '전용': 3, '만': 3, '소': 3, '김': 3, '국내': 2, '수입': 2, '개발': 2, '위': 2, '현대': 2, '계획': 2, '카': 2, '공장': 2, '내연': 2, '현대차': 2, '대': 2, '전망': 2, '최근': 2, '비전': 2, '지향': 2, '전국': 2, '정도': 2, '주행': 2, '관련': 2, '융합': 2, '기관': 2, '설명': 2, '삼성': 2, '만남': 2, '의미': 2, '부분': 2, '배터리': 2, '엔트리': 2, '이용': 2, '상황': 1, '회사': 1, '박차': 1, '본격': 1, '포니': 1, '모티브': 1, '콘셉트': 1, '스타일': 1, '플랫': 1, '폼': 1, '확보': 1, '울산': 1, '기관차': 1, '일부': 1, '교체': 1, '중': 1, '노조': 1, '등': 1, '판매': 1, '부문': 1, '유지': 1, '메르세데스': 1, '벤츠': 1, '탄소': 1, '중립': 1, '달성': 1, '목표': 1, '역량': 1, '집중': 1, '뉴': 1, '공개': 1, '전략': 1, '발표': 1, '성공': 1, '선점': 1, '시공간': 1, '보조': 1, '예정': 1, '소도': 1, '급증': 1, '환경부': 1, '조사': 1, '결과': 1, '집계': 1, '전': 1, '비교': 1, '배': 1, '이상': 1, '충': 1, '전소': 1, '곳': 1, '경기도': 1, '보유': 1, '다음': 1, '제주': 1, '번': 1, '서울': 1, '지역': 1, '내': 1, '파악': 1, '이':

In [32]:
print(df_air_pol['nouns_counter'][1])

Counter({'친환경': 11, '칼': 10, '텍스': 10, '경영': 8, '회장': 6, '일': 5, '월': 5, '화학': 5, '배출': 5, '사고': 4, '디지털': 4, '이후': 4, '것': 4, '여수': 4, '량': 4, '재발': 4, '방지': 4, '코로나': 3, '포럼': 3, '등': 3, '폐수': 3, '유출': 3, '대기': 3, '오염': 3, '물질': 3, '조작': 3, '노력': 3, '년': 3, '그룹': 2, '강조': 2, '포스트': 2, '시대': 2, '대비': 2, '계열사': 2, '마련': 2, '촉구': 2, '역량': 2, '우리': 2, '향후': 2, '이번': 2, '확대': 2, '혁신': 2, '전환': 2, '핵심': 2, '실천': 2, '홈': 2, '쇼핑': 2, '시설': 2, '연료': 2, '배송': 2, '도입': 2, '지난해': 2, '사건': 2, '산단': 2, '토양': 2, '간': 2, '삼남': 2, '석유': 2, '화재': 2, '시민': 2, '관리': 2, '수장': 1, '허태수': 1, '미래': 1, '설계': 1, '발': 1, '대응책': 1, '거리': 1, '엇박자': 1, '서울': 1, '남촌': 1, '리더십': 1, '센터': 1, '임원': 1, '최근': 1, '글로벌': 1, '공급': 1, '망': 1, '재편': 1, '모바일': 1, '활성': 1, '경제': 1, '확산': 1, '강화': 1, '부족': 1, '점': 1, '고도': 1, '계기': 1, '당부': 1, '취임': 1, '번': 1, '회의': 1, '팬데': 1, '믹': 1, '사업': 1, '전략': 1, '점검': 1, '위기': 1, '극복': 1, '방안': 1, '머리': 1, '전임': 1, '허창수': 1, '용퇴': 1, '리더': 1, '낙점': 1, '작업': 1, '속도': 1, '키워드': 1, '지속': 1

In [33]:
print(df_air_pol['nouns_counter'][2])

Counter({'사업': 3, '한국전력공사': 2, '인도네시아': 2, '화력': 2, '발전소': 2, '환경': 2, '등': 2, '자바': 1, '호기': 1, '건설': 1, '추진': 1, '일': 1, '서울': 1, '서초구': 1, '한전': 1, '아트': 1, '센터': 1, '사회': 1, '해당': 1, '안건': 1, '의결': 1, '보류': 1, '정부': 1, '국가': 1, '전력': 1, '인프라': 1, '사업비': 1, '억': 1, '달러': 1, '이': 1, '자카르타': 1, '인근': 1, '규모': 1, '석탄': 1, '기': 1, '이번': 1, '결정': 1, '정치': 1, '단체': 1, '수익': 1, '논란': 1, '온실가스': 1, '생산': 1, '오염': 1, '부정': 1, '여론': 1, '반영': 1, '것': 1, '풀이': 1})


In [34]:
print(df_air_pol['nouns_counter'][3])

Counter({'일': 2, '공단': 2, '도로': 2, '물': 2, '포스코': 1, '현대제철': 1, '동국제강': 1, '에코': 1, '시스템': 1, '제철': 1, '세라믹': 1, '세기': 1, '등': 1, '개': 1, '회사': 1, '폭염': 1, '비산': 1, '먼지': 1, '대책': 1, '일환': 1, '철강': 1, '관리': 1, '내': 1, '주요': 1, '작업': 1, '업체': 1, '지원': 1, '살수차': 1, '포항시': 1, '제공': 1, '경북': 1, '매일': 1})


In [35]:
print(df_air_pol['nouns_counter'][4])

Counter({'대기': 8, '측정': 8, '오염': 7, '영풍': 6, '제련소': 6, '환경': 6, '물질': 5, '재판부': 5, '씨': 4, '징역': 3, '실형': 3, '선고': 3, '등': 3, '법': 3, '조작': 2, '년': 2, '개월': 2, '이사': 2, '모': 2, '집행': 2, '방해': 2, '대행업체': 2, '현대': 2, '공해': 2, '주식회사': 2, '이': 2, '박': 2, '이미지': 2, '재고': 2, '공모': 2, '결과': 2, '업무': 2, '교체': 2, '보전': 2, '위반': 2, '부과금': 2, '측정치': 1, '구속': 1, '기소': 1, '석포': 1, '임원': 1, '일': 1, '대구': 1, '지방': 1, '법원': 1, '서부': 1, '지원': 1, '형사': 1, '단독': 1, '위계': 1, '공무': 1, '혐의': 1, '사내': 1, '사': 1, '월': 1, '삼안': 1, '화학': 1, '관계자': 1, '명': 1, '집행유예': 1, '분석': 1, '환경부': 1, '점검': 1, '대비': 1, '하드': 1, '디스크': 1, '담당자': 1, '휴대': 1, '전화': 1, '증거': 1, '인멸': 1, '것': 1, '총괄': 1, '과거': 1, '토양': 1, '차례': 1, '벌금형': 1, '처벌': 1, '전력': 1, '점': 1, '반영': 1, '기본': 1, '면제': 1, '관리': 1, '대외': 1, '거짓': 1, '기록': 1, '담당': 1, '공무원': 1, '집무': 1, '범행': 1, '장기간': 1, '조직': 1, '계획': 1, '정도': 1, '지적': 1, '법률': 1, '준수': 1, '개선책': 1, '마련': 1, '노력': 1, '수치': 1, '자료': 1, '삭제': 1, '배출': 1, '제도': 1, '신뢰': 1, '저하': 1, '악화': 1, '훼손'

In [36]:
print(df_air_pol['nouns_counter'][5])

Counter({'포항': 2, '제철소': 2, '년': 2, '봉화': 1, '영풍': 1, '석포': 1, '제련소': 1, '포스코': 1, '조업': 1, '정지': 1, '행정': 1, '처분': 1, '상황': 1, '영업': 1, '허가': 1, '이후': 1, '수십': 1, '간': 1, '대기': 1, '오염': 1, '물질': 1})


In [37]:
print(df_air_pol['nouns_counter'][6])

Counter({'현대제철': 4, '제철소': 4, '대기': 4, '시설': 4, '당진': 3, '오염': 3, '물질': 3, '것': 3, '일': 3, '방지': 2, '중': 2, '배출': 2, '신고': 2, '고로': 2, '운영': 2, '등': 2, '점검': 2, '설치': 2, '채': 1, '고장': 1, '방치': 1, '이': 1, '충남': 1, '조업': 1, '정지': 1, '사용': 1, '중지': 1, '처분': 1, '검찰': 1, '고발': 1, '충': 1, '남도': 1, '도': 1, '당진시': 1, '경기도': 1, '환경': 1, '단체': 1, '관계자': 1, '명': 1, '도내': 1, '사업장': 1, '특별': 1, '합동': 1, '결과': 1, '용광': 1, '정비': 1, '과정': 1, '발생': 1, '먼지': 1, '새벽': 1, '시간': 1, '연주': 1, '공정': 1, '기타로': 1, '확인': 1})


In [38]:
print(df_air_pol['nouns_counter'][7])

Counter({'공기': 9, '제품': 7, '청정': 6, '전자': 5, '케어': 5, '청정기': 5, '퓨리': 4, '대형': 4, '구매': 4, '설치': 3, '미세먼지': 3, '관리': 3, '스티커': 3, '수': 3, '면적': 3, '협회': 3, '공간': 3, '알레르기': 3, '지역': 2, '부착': 2, '매장': 2, '중': 2, '고객': 2, '상업': 2, '성능': 2, '한국': 2, '인증': 2, '상업시설': 1, '공공시설': 1, '홍보': 1, '일': 1, '제공': 1, '구입': 1, '경우': 1, '신청': 1, '안내': 1, '문구': 1, '조사': 1, '결과': 1, '이상': 1, '업소': 1, '등': 1, '용도': 1, '이유': 1, '응답': 1, '후': 1, '만족': 1, '점': 1, '제곱미터': 1, '가정': 1, '부족': 1, '측': 1, '설명': 1, '흡입구': 1, '토출구': 1, '앞쪽': 1, '뒷면': 1, '옆면': 1, '별도': 1, '필요': 1, '벽': 1, '밀착': 1, '때문': 1, '활용': 1, '단계': 1, '토탈': 1, '플러스': 1, '필터': 1, '시스템': 1, '초미세': 1, '먼지': 1, '이산화': 1, '황': 1, '이산화질소': 1, '톨루엔': 1, '포름': 1, '데이드': 1, '유해': 1, '물질': 1, '천식': 1, '영국': 1, '질환': 1, '예방': 1, '환경': 1, '획득': 1, '백승태': 1, '사업': 1, '담당': 1, '상무': 1, '향상': 1, '실내': 1, '차별': 1, '경쟁력': 1, '확보': 1, '지속': 1, '지원': 1, '말': 1})


In [39]:
print(df_air_pol['nouns_counter'][8])

Counter({'선고': 8, '것': 8, '법': 7, '월': 6, '일': 6, '주장': 6, '피고인': 6, '유화': 5, '환경': 5, '배출': 5, '등': 5, '산': 4, '공장': 4, '처벌': 4, '오염': 4, '벌금': 4, '재판부': 4, '대기': 4, '플레어': 4, '스택': 4, '재판': 4, '시설': 4, '경고': 3, '메시지': 3, '공장장': 3, '범행': 3, '법정': 3, '기업': 3, '이익': 3, '씨': 3, '면소': 3, '비산': 3, '이': 3, '로그인': 3, '실형': 2, '최고액': 2, '울산': 2, '지법': 2, '차후': 2, '예방': 2, '지난해': 2, '매연': 2, '구속': 2, '해당': 2, '설명': 2, '보전': 2, '혐의': 2, '만': 2, '시': 2, '분': 2, '처리': 2, '초과': 2, '유입': 2, '조항': 2, '반성': 2, '폐지': 2, '규제': 2, '무죄': 2, '개정': 2, '시행': 2, '이유': 2, '범죄': 2, '삭제': 2, '범위': 2, '필요': 2, '물질': 2, '가능': 2, '기사': 2, '댓글': 2, '이야기': 2, '후': 2, '사회': 1, '종합': 1, '강력': 1, '차원': 1, '승인': 1, '굴뚝': 1, '화염': 1, '업체': 1, '여긴': 1, '책임자': 1, '위반': 1, '기소': 1, '징역': 1, '상한선': 1, '원': 1, '오후': 1, '사업장': 1, '용량': 1, '에틸렌': 1, '프로필렌': 1, '벤젠': 1, '탄화수소': 1, '차례': 1, '발생': 1, '사건': 1, '형벌': 1, '조': 1, '호': 1, '고려': 1, '방지': 1, '관리': 1, '만큼': 1, '중처': 1, '년': 1, '때문': 1, '근거': 1, '행정': 1, '이중': 1, '독자': 1, 

In [40]:
print(df_air_pol['nouns_counter'][9])

Counter({'영풍': 10, '제련소': 7, '주장': 6, '것': 6, '석포': 5, '환경': 5, '위원장': 5, '오염': 4, '이': 4, '공장': 4, '제련': 4, '등': 4, '설치': 4, '일': 3, '사실': 3, '조업': 3, '량': 3, '밤': 3, '근거': 3, '공정': 3, '배출': 3, '수': 3, '봉화군': 2, '보도': 2, '주민': 2, '건강': 2, '피해': 2, '대책': 2, '허위': 2, '유포': 2, '대응': 2, '낮': 2, '가스': 2, '뿐': 2, '기술': 2, '가능': 2, '의도': 2, '대량': 2, '식': 2, '설명': 2, '굴뚝': 2, '물질': 2, '측정': 2, '법령': 2, '거대': 2, '낙동강': 2, '봉화': 1, '경북': 1, '소재': 1, '대구': 1, '신문': 1, '정수근': 1, '공동': 1, '법': 1, '자료': 1, '꼼수': 1, '하늘': 1, '괴담': 1, '시설': 1, '대대': 1, '밤중': 1, '유해': 1, '범죄': 1, '행위': 1, '카더라': 1, '통신': 1, '인용': 1, '회사': 1, '종사자': 1, '신뢰도': 1, '명예': 1, '침해': 1, '그': 1, '응분': 1, '책임': 1, '병': 1, '년': 1, '전': 1, '일본': 1, '광산': 1, '발생': 1, '사고': 1, '폭거': 1, '지속': 1, '방': 1, '류구': 1, '시간': 1, '실시간': 1, '공단': 1, '전송': 1, '장비': 1, '관련': 1, '대상': 1, '항목': 1, '관리': 1, '임의': 1, '여부': 1, '결정': 1, '강조': 1, '정': 1, '측': 1, '불법': 1, '일관': 1, '특성': 1, '가동': 1, '업': 1, '단일': 1, '기업': 1, '산업': 1, '언어': 1, '폭력': 1, '

In [41]:
print(df_air_pol['nouns_counter'][10])

Counter({'포천': 14, '고발': 8, '에너지': 7, '발전소': 7, '가동': 7, '신평': 7, '리': 7, '단체': 6, '법': 6, '계획': 6, '이': 6, '시민': 5, '사업': 5, '석탄': 5, '그린': 5, '너지': 5, '환경영향평가': 5, '사용': 5, '지역': 4, '공동': 4, '포천시': 4, '주민': 4, '월': 4, '상업': 4, '일원': 4, '이행': 4, '일': 4, '승인': 4, '관련': 4, '변경': 4, '집단': 3, '위반': 3, '대응': 3, '반대': 3, '투쟁': 3, '굴뚝': 3, '협의': 3, '내용': 3, '시': 3, '상황': 3, '것': 3, '입장': 3, '혐의': 2, '확인': 2, '본부': 2, '투데이': 2, '신문': 2, '경기도': 2, '화력': 2, '시작': 2, '산업단지': 2, '등': 2, '대기': 2, '오염': 2, '우려': 2, '석투': 2, '연합회': 2, '환경': 2, '산업': 2, '통상': 2, '자원': 2, '부': 2, '시설': 2, '보일러': 2, '문제': 2, '불법': 2, '행위': 2, '지적': 2, '회견': 2, '허가': 2, '준공': 2, '건축법': 2, '근거': 2, '수': 2, '고발장': 2, '예정': 1, '여부': 1, '지난해': 1, '반발': 1, '본': 1, '미비': 1, '서류': 1, '보완': 1, '방침': 1, '준비': 1, '위원회': 1, '포': 1, '천': 1, '자치': 1, '한국': 1, '기후': 1, '네트워크': 1, '시이': 1, '통장': 1, '개': 1, '동참': 1, '년': 1, '공사': 1, '해': 1, '말': 1, '착공': 1, '작년': 1, '본격': 1, '돌입': 1, '신북면': 1, '무허가': 1, '염색집': 1, '단화': 1, '단지': 1, '개별'

In [42]:
print(df_air_pol['nouns_counter'][11])

Counter({'포스코': 11, '공사': 11, '건설': 7, '포항': 6, '고발': 4, '일': 4, '홍': 4, '보관': 4, '문화': 4, '조성': 4, '경찰': 3, '시설': 3, '설치': 3, '포항시': 3, '배짱': 2, '세': 2, '대': 2, '환경': 2, '보전': 2, '법': 2, '위반': 2, '조치': 2, '이행': 2, '이': 2, '적발': 2, '층': 2, '현장': 1, '수모': 1, '대기': 1, '비산': 1, '먼지': 1, '억제': 1, '미': 1, '혐의': 1, '남부': 1, '경찰서': 1, '시': 1, '공사장': 1, '수': 1, '변경': 1, '신고': 1, '사후': 1, '남구': 1, '괴동동': 1, '일원': 1, '연면적': 1, '건축': 1, '면적': 1, '총': 1, '사업': 1, '부지': 1, '만': 1, '지하': 1, '지상': 1, '규모': 1, '내년': 1, '월': 1, '준공': 1})


In [43]:
print(df_air_pol['nouns_counter'][12])

Counter({'가능': 2, '롯데케미칼': 1, '측': 1, '해당': 1, '소각': 1, '시설': 1, '허용': 1, '기준': 1, '초과': 1, '고농도': 1, '암모니아': 1, '배출': 1, '오류': 1, '개입': 1, '배제': 1, '수': 1, '주장': 1, '법원': 1})


In [44]:
print(df_air_pol['nouns_counter'][13])

Counter({'발전': 14, '대기': 12, '오염': 11, '물질': 11, '지역': 11, '한국': 10, '중부': 10, '발전소': 9, '등': 9, '감축': 8, '환경': 7, '석탄': 7, '미세먼지': 6, '억': 5, '설비': 5, '배출': 5, '사회': 5, '지원': 5, '투자': 5, '년': 5, '전환': 5, '량': 4, '저감': 4, '원': 4, '운영': 4, '연료': 4, '개': 4, '개선': 3, '계층': 3, '노력': 3, '지난해': 3, '보령': 3, '성능': 3, '먼지': 3, '전력': 3, '계획': 3, '로드맵': 3, '중소기업': 3, '사업': 3, '주변': 3, '최신': 2, '국민': 2, '건강': 2, '취약': 2, '적극': 2, '포함': 2, '설치': 2, '대비': 2, '만': 2, '확대': 2, '호기': 2, '보강': 2, '최고': 2, '수준': 2, '저탄장': 2, '비산': 2, '발생': 2, '선박': 2, '하역': 2, '필요': 2, '대체': 2, '기존': 2, '보완': 2, '유류': 2, '수립': 2, '건설': 2, '반영': 2, '데': 2, '청정': 2, '자금': 2, '노인': 2, '대': 2, '진공': 2, '청소차': 2, '소통': 2, '정보': 2, '공개': 2, '개소': 2, '총력': 1, '협력': 1, '방출': 1, '동시': 1, '결실': 1, '상당': 1, '기준': 1, '저유': 1, '황탄': 1, '비중': 1, '호': 1, '탈황': 1, '용량': 1, '주효': 1, '세계': 1, '도입': 1, '황산화물': 1, '질소': 1, '산화물': 1, '국내': 1, '해외': 1, '입증': 1, '차단': 1, '옥내': 1, '추진': 1, '유연탄': 1, '수송': 1, '때': 1, '생산': 1, '과정': 1, '이': 1, '하

In [52]:
print(df_air_pol['nouns_counter'][14])

Counter({'배출': 11, '오염': 8, '초과': 8, '대기': 7, '사업장': 6, '등': 6, '물질': 5, '충남': 5, '부과금': 5, '현대': 4, '천안': 4, '환경': 4, '상반기': 3, '당진': 3, '것': 3, '굴뚝': 3, '자동': 3, '측정': 3, '기기': 3, '부착': 3, '부과': 3, '사업': 3, '곳': 3, '현대제철': 2, '뱅크': 2, '지역': 2, '개소': 2, '중': 2, '억': 2, '만': 2, '원': 2, '일': 2, '이': 2, '개': 2, '서산': 2, '공장': 2, '장': 2, '상하': 2, '반기': 2, '전경': 1, '사진': 1, '대전일보': 1, '오일': 1, '주요': 1, '도내': 1, '년': 1, '가운데': 1, '아산': 1, '서': 1, '북부': 1, '위치': 1, '확인': 1, '오': 1, '미칼': 1, '코스모': 1, '영흥': 1, '산업': 1, '삼광': 1, '글라스': 1, '유리': 1, '삼정': 1, '펄프': 1, '지에스': 1, '피에스': 1, '허용': 1, '기준': 1, '제도': 1, '항목': 1, '질소': 1, '산화물': 1, '황산화물': 1, '염화수소': 1, '먼지': 1, '량': 1, '산정': 1, '대상': 1, '진행': 1, '지난해': 1, '저감': 1, '방지': 1, '기금': 1, '재원': 1, '사용': 1, '관계자': 1, '총량': 1, '관리제': 1, '시행': 1, '전망': 1, '도민': 1, '건강': 1, '안전': 1, '정책': 1, '발굴': 1, '말': 1})


In [53]:
print(df_air_pol['nouns_counter'][15])

Counter({'배출': 13, '암모니아': 8, '롯데케미칼': 7, '초과': 7, '시설': 6, '기준': 6, '대기': 5, '소각': 5, '측정': 5, '해당': 5, '오염': 4, '물질': 4, '부과금': 4, '부과': 4, '전남': 4, '시험': 4, '처분': 3, '소송': 3, '보건': 3, '환경': 3, '연구원': 3, '여수': 3, '공장': 3, '조사': 3, '측': 3, '공정': 3, '가능': 3, '전라남도': 2, '상대': 2, '취소': 2, '제기': 2, '패소': 2, '혐의': 2, '법원': 2, '폐가스': 2, '허용': 2, '행정': 2, '농도': 2, '법': 2, '사용': 2, '등': 2, '수': 2, '광주': 2, '주장': 2, '재판부': 2, '년': 2, '점': 2, '것': 1, '게': 1, '판단': 1, '지난해': 1, '월': 1, '개선': 1, '명령': 1, '천': 1, '만': 1, '원': 1, '시료': 1, '채취': 1, '배': 1, '출원': 1, '량': 1, '실시': 1, '적서': 1, '근거': 1, '과정': 1, '절차': 1, '위반': 1, '결과': 1, '신뢰': 1, '일': 1, '지역': 1, '언론사': 1, '법조': 1, '지법': 1, '부': 1, '지사': 1, '원고': 1, '판결': 1, '고농도': 1, '오류': 1, '개입': 1, '배제': 1, '이유': 1, '기각': 1, '사건': 1, '판시': 1, '회사': 1, '양': 1, '등록': 1, '포함': 1, '취지': 1, '변경': 1, '신고': 1, '감안': 1})


In [54]:
print(df_air_pol['nouns_counter'][16])

Counter({'화학': 2, '전남': 1, '여수시': 1, '의회': 1, '대기': 1, '오염': 1, '물질': 1, '측정': 1, '대행업체': 1, '배출': 1, '량': 1, '조작': 1, '칼': 1, '텍스': 1, '금호': 1, '석유': 1, '롯데케미칼': 1, '한화케미칼': 1, '등': 1, '여수': 1, '산단': 1, '입주': 1, '기업': 1, '사회': 1})


In [55]:
print(df_air_pol['nouns_counter'][17])

Counter({'도료': 30, '무용': 19, '친환경': 17, '선박': 15, '도장': 15, '기술': 14, '개발': 10, '작업': 9, '확보': 8, '수': 7, '제품': 7, '사용': 7, '환경': 7, '오염': 6, '용제': 6, '대기': 6, '조선소': 6, '때': 5, '위험': 5, '조선': 5, '경쟁력': 4, '적용': 4, '업계': 4, '시장': 4, '등': 4, '것': 4, '한국': 4, '현대': 4, '중공업': 4, '시설': 4, '상용': 3, '국내': 3, '어려움': 3, '때문': 3, '화재': 3, '사고': 3, '유기': 3, '이상': 3, '물': 3, '진행': 3, '차별': 2, '예정': 2, '가운데': 2, '연구': 2, '올해': 2, '규모': 2, '물질': 2, '배출': 2, '조건': 2, '극복': 2, '최근': 2, '해양': 2, '한계': 2, '이': 2, '표준': 2, '세계': 2, '경우': 2, '보전': 2, '법': 2, '시행': 2, '개정안': 2, '방지': 2, '설치': 2, '년': 2, '확대': 2, '유지': 2, '중': 2, '가사': 2, '시간': 2, '일부': 2, '폭발': 2, '질식': 2, '개': 2, '프로젝트': 2, '사진': 1, '동일': 1, '선종': 1, '폴라리스': 1, '핑': 1, '광석선': 1, '페인트': 1, '분야': 1, '열기': 1, '활기': 1, '조선사': 1, '선점': 1, '속도': 1, '각종': 1, '미래': 1, '힘': 1, '상황': 1, '억': 1, '원': 1, '정도': 1, '추정': 1, '고품질': 1, '날개': 1, '여무': 1, '건조': 1, '균일': 1, '품질': 1, '그늘': 1, '시대': 1, '주인공': 1, '기간': 1, '데': 1, '성공': 1, '신조선': 1, '사양': 1, '

In [56]:
print(df_air_pol['nouns_counter'][18])

Counter({'선박': 9, '연료': 5, '조선': 5, '개발': 5, '암모니아': 4, '년': 4, '규제': 4, '등': 4, '친환경': 4, '배출': 3, '감축': 3, '선도': 3, '추진': 3, '현대': 3, '미포': 3, '기술': 3, '수': 3, '수소': 2, '탄소': 2, '대체': 2, '온실가스': 2, '량': 2, '강화': 2, '해양': 2, '공동': 2, '정보': 2, '체결': 2, '관계자': 2, '것': 2, '연구': 2, '제품': 2, '질소': 1, '합성': 1, '화합물': 1, '시': 1, '이산화': 1, '공급': 1, '안정': 1, '보관': 1, '운송': 1, '취급': 1, '비교': 1, '시대': 1, '중': 1, '하나': 1, '주목': 1, '국제': 1, '대비': 1, '최소': 1, '이상': 1, '시행': 1, '걸음': 1, '논의': 1, '예정': 1, '한국': 1, '자회사': 1, '현대미': 1, '포조': 1, '로이드': 1, '선급': 1, '상용': 1, '목표': 1, '프로젝트': 1, '착수': 1, '선': 1, '기본': 1, '설계': 1, '울산': 1, '산업': 1, '흥원': 1, '통신': 1, '융합': 1, '전기': 1, '스마트': 1, '척': 1, '건': 1, '조': 1, '계약': 1, '시장': 1, '공략': 1, '박차': 1, '선제': 1, '대응': 1, '역량': 1, '진행': 1, '분야': 1, '강조': 1, '대우': 1, '올해': 1, '한화': 1, '디펜스': 1, '리튬': 1, '배터리': 1, '기반': 1, '에너지': 1, '저장': 1, '장치': 1, '협약': 1, '양사': 1, '내': 1, '발전기': 1, '전력': 1, '부하': 1, '최적': 1, '상태': 1, '제어': 1, '운항': 1, '발생': 1, '오염': 1, '물질'

In [75]:
print(df_air_pol['nouns_counter'][19])

Counter({'배출': 27, '량': 19, '화학': 11, '물질': 9, '측정': 9, '방식': 7, '산정': 7, '방법': 7, '조사': 7, '년': 6, '등': 6, '것': 6, '조작': 5, '환경': 4, '개선': 4, '실제': 4, '결과': 4, '당국': 3, '작업': 3, '업체': 3, '수': 3, '정확': 3, '오염': 3, '계수법': 3, '여수': 2, '산업단지': 2, '곳': 2, '간': 2, '제도': 2, '지적': 2, '석유': 2, '안전원': 2, '관계자': 2, '사업장': 2, '핵심': 2, '차이': 2, '단위': 2, '공정': 2, '계수': 2, '중': 2, '때문': 2, '지난해': 2, '대기': 2, '재발': 2, '확대': 2, '단체': 2, '중심': 2, '법': 2, '전남': 1, '굴뚝': 1, '연기': 1, '작년': 1, '수백': 1, '집단': 1, '사실': 1, '이번': 1, '실질': 1, '대안': 1, '도출': 1, '근절': 1, '주목': 1, '일': 1, '정유': 1, '업계': 1, '환경부': 1, '산하': 1, '최근': 1, '착수': 1, '기준': 1, '마련': 1, '게': 1, '정도': 1, '상황': 1, '반영': 1, '신뢰': 1, '차원': 1, '설명': 1, '사업': 1, '보완': 1, '추가': 1, '검토': 1, '재산': 1, '수란': 1, '하나': 1, '배': 1, '출원': 1, '연간': 1, '조업': 1, '시간': 1, '간접': 1, '만큼': 1, '직접': 1, '측': 1, '정법': 1, '단점': 1, '자료': 1, '분석': 1, '경제': 1, '협력': 1, '개발': 1, '기구': 1, '국가': 1, '채택': 1, '비교': 1, '계획': 1, '이': 1, '지금': 1, '사이': 1, '괴리': 1, '사건': 1, '문제'

In [88]:
print(df_air_pol['nouns_counter'][20])

Counter({'추진': 10, '선': 10, '현대': 9, '연료': 8, '중공업': 7, '선박': 6, '가스': 6, '엔진': 6, '다라마': 6, '바사': 6, '공급': 5, '친환경': 5, '가나': 5, '사업': 4, '시스템': 4, '대형': 3, '설비': 3, '기술': 3, '것': 3, '이중': 3, '월': 3, '수': 3, '국내': 2, '양산': 2, '설치': 2, '완료': 2, '확대': 2, '성장': 2, '발판': 2, '마련': 2, '건조': 2, '본격': 2, '구축': 2, '미포': 2, '조선': 2, '급': 2, '디젤': 2, '사용': 2, '대비': 2, '올해': 2, '만': 2, '최초': 1, '청정': 1, '확장': 1, '중': 1, '이미지': 1, '최근': 1, '울산': 1, '공장': 1, '부품': 1, '조립': 1, '시운전': 1, '수행': 1, '이번': 1, '처음': 1, '일': 1, '마력': 1, '시작': 1, '시': 1, '운전': 1, '때': 1, '황산화물': 1, '배출': 1, '대기': 1, '오염': 1, '방지': 1, '규칙': 1, '장점': 1, '기체': 1, '액화': 1, '별도': 1, '기화기': 1, '필요': 1, '저장': 1, '오랫동안': 1, '개발': 1, '심혈': 1, '지금': 1, '척': 1, '수주': 1, '삼': 1, '호': 1, '착공': 1, '바': 1, '미국': 1, '선급': 1, '협회': 1, '리아': 1, '베리아': 1, '기국': 1, '협약': 1, '독자': 1, '화물': 1, '운영': 1, '국산': 1, '연계': 1, '재액': 1, '효율': 1, '해외': 1, '제품': 1, '구성': 1, '단순': 1, '원가': 1, '경쟁력': 1, '강화': 1, '세계': 1, '벙커': 1, '시설': 1, '가능': 1, '선종': 1, 

## 단어들의 Intersection 확인

### 뉴스 기사에 나온 단어들이 얼마나 겹치는 지 확인

In [46]:
check_list = list(set(a) for a in df_air_pol['contents_nouns_len>1'][0:4])
result = reduce(set.intersection, check_list)

In [47]:
result

set()

**뉴스기사 전체집합의 입장에서 보면 뉴스들에 나오는 단어들은 거의 안 겹친다.**

### 특정 단어가 들어간 뉴스 기사 확인

In [113]:
corpus =[]
for a in df_air_pol['contents_nouns']:
    corpus.append(a)

In [114]:
len(flattened_list(corpus))

55505

In [115]:
corpus = list(set(flattened_list(corpus)))

In [116]:
len(corpus)

4578

In [117]:
get_word_pct_dict(corpus)

[('환경', 0.7098039215686275),
 ('오염', 0.6823529411764706),
 ('배출', 0.6823529411764706),
 ('대기', 0.6784313725490196),
 ('물질', 0.6431372549019608),
 ('미세먼지', 0.5333333333333333),
 ('관리', 0.4588235294117647),
 ('저감', 0.44313725490196076),
 ('지역', 0.4392156862745098),
 ('개선', 0.43529411764705883),
 ('기업', 0.40784313725490196),
 ('사업', 0.38823529411764707),
 ('시설', 0.38823529411764707),
 ('계획', 0.3843137254901961),
 ('발생', 0.3803921568627451),
 ('설치', 0.3607843137254902),
 ('추진', 0.3568627450980392),
 ('관계자', 0.3568627450980392),
 ('운영', 0.3411764705882353),
 ('지난해', 0.3215686274509804),
 ('측정', 0.30980392156862746),
 ('이번', 0.30980392156862746),
 ('발전', 0.30196078431372547),
 ('결과', 0.30196078431372547),
 ('기준', 0.2980392156862745),
 ('환경부', 0.2980392156862745),
 ('사업장', 0.2980392156862745),
 ('먼지', 0.29411764705882354),
 ('친환경', 0.29411764705882354),
 ('진행', 0.2823529411764706),
 ('설비', 0.27450980392156865),
 ('한국', 0.27058823529411763),
 ('지속', 0.27058823529411763),
 ('사용', 0.266666666666